<a href="https://colab.research.google.com/github/Sazahiba/AssignmentQ2/blob/main/Flowers_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import pathlib
from pathlib import Path

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Loading Data And Defining Features And Labels
directory = Path("/content/drive/MyDrive/DL Data Sets/flowers")

image_names = []
data_images = []
labels = []

# Iterating Over Directory To Extract Sub Directories
for dir in directory.iterdir():
  image_names.append(dir.name)
  print(dir.name)
# Iterating Over Sub Directories To Extract Lables
  for imgpath in dir.iterdir():
    if imgpath.name.endswith("jpg"):
      labels.append(dir.name)
      imgarr = cv2.imread(str(imgpath), cv2.IMREAD_GRAYSCALE)
      imgarr = cv2.resize(imgarr, (320,240))
      data_images.append(imgarr)

rose
dandelion
daisy
flowers
sunflower
tulip


In [4]:
# maping the string label into numeric 
df=pd.DataFrame(labels)
labels=df[0].map({"rose":0,"sunflower":1,"tulip":2,"daisy":3,"dandelion":4})

In [5]:
# data and label spliting by machine learning libarary
import sklearn
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data_images,labels,test_size=.40,random_state=0)

In [6]:
#convert data into numpy array and data type float
X_train=np.asarray(X_train).astype("float32")
X_test=np.asarray(X_test).astype("float32")

In [7]:
# Check the shape of training data,testing data,training labels,testing labels
print(f"shape of training data is :{X_train.shape}\nshape of training label is :{y_train.shape}\nshape of testing data is :{X_test.shape}\nshape of testing label is :{y_test.shape}")

shape of training data is :(2593, 240, 320)
shape of training label is :(2593,)
shape of testing data is :(1730, 240, 320)
shape of testing label is :(1730,)


In [8]:
#reshaping
X_train=X_train.reshape(2593,320*240)
X_test=X_test.reshape(1730,320*240)

In [9]:
#check value of train data
X_train[0]

array([255., 255., 255., ..., 121., 125., 125.], dtype=float32)

In [10]:
#check value of test data
X_test[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [11]:
#normalize the data
X_train=X_train/255.0
X_test=X_test/255.0

In [14]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [15]:
# importing keras libararies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop

In [16]:
#model building
network=Sequential()
network.add(Dense(256,activation="relu",input_shape=(X_train.shape[1],)))
network.add(Dropout(0.2))
network.add(Dense(128,activation="relu"))
#network.add(Dropout(0.2))
network.add(Dense(64,activation="relu"))
network.add(Dropout(0.2))
network.add(Dense(5,activation="softmax"))

In [17]:
#model summary
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               19661056  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 19,702,533
Trainable params: 19,702,533
Non-trainable params: 0
____________________________________________

In [18]:
# Compilation Step
network.compile(optimizer=RMSprop(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [19]:
history=network.fit(X_train, y_train,batch_size=20,epochs=100)

Epoch 1/100
130/130 [==============================] - 22s 160ms/step - loss: 2.8506 - accuracy: 0.2227
Epoch 2/100
130/130 [==============================] - 21s 159ms/step - loss: 1.6789 - accuracy: 0.2511
Epoch 3/100
130/130 [==============================] - 21s 159ms/step - loss: 1.6194 - accuracy: 0.2420
Epoch 4/100
130/130 [==============================] - 21s 159ms/step - loss: 1.5719 - accuracy: 0.2704
Epoch 5/100
130/130 [==============================] - 21s 160ms/step - loss: 1.5848 - accuracy: 0.2733
Epoch 6/100
130/130 [==============================] - 21s 159ms/step - loss: 1.5631 - accuracy: 0.2892
Epoch 7/100
130/130 [==============================] - 21s 160ms/step - loss: 1.5432 - accuracy: 0.2804
Epoch 8/100
130/130 [==============================] - 21s 160ms/step - loss: 1.5560 - accuracy: 0.2909
Epoch 9/100
130/130 [==============================] - 21s 159ms/step - loss: 1.5608 - accuracy: 0.2744
Epoch 10/100
130/130 [==============================] - 21s 159m

In [20]:
history_dict = history.history
history_dict.keys()
[u'accuracy', u'loss']

['accuracy', 'loss']

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss']
val_accuracy = history_dict['accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)
plt.plot(epochs, loss_values, 'r', label='loss')
plt.plot(epochs, val_accuracy, 'b', label='accuracy')
plt.title('loss and accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [21]:
network.evaluate(X_test,y_test)

55/55 [==============================] - 2s 33ms/step - loss: 1.7919 - accuracy: 0.3040


[1.7919350862503052, 0.30404624342918396]

In [22]:
pred = network.predict_classes(X_test[:10])
for i in range(len(pred)):
    print(pred[i],'==>',y_test[i])

0 ==> [0. 0. 0. 1. 0.]
1 ==> [0. 0. 1. 0. 0.]
1 ==> [0. 0. 1. 0. 0.]
4 ==> [0. 0. 0. 0. 1.]
4 ==> [0. 0. 0. 0. 1.]
4 ==> [0. 0. 1. 0. 0.]
2 ==> [0. 1. 0. 0. 0.]
2 ==> [0. 1. 0. 0. 0.]
1 ==> [0. 0. 1. 0. 0.]
2 ==> [1. 0. 0. 0. 0.]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
